CS524: Introduction to Optimization Lecture 19
======================================

## Michael Ferris<br> Computer Sciences Department <br> University of Wisconsin-Madison

## October 18, 2023
--------------

In [1]:
%load_ext gams.magic
m = gams.exchange_container

#  Gandhi (Winston, p 463)

A clothing manufacturer can make shirts, shorts, and pants. A different machine must be rented to make each of these three items. A shirt machine costs \\$200/month to rent, while a shorts machine costs \\$150/month and a pants machine costs \\$100/month. (The manufacturer can choose not to rent a shirt machine, for example, but in this case he cannot make any shirts.) Each shirt requires 3 hours of labor and 4 square yards of cloth; each pair of pants requires 6 hours of labor and 4 square yards of cloth; each pair of shorts requires 2 hours of labor and 3 square yards of cloth. The total amount of labor available is 150 hours, and there are 160 square yards of cloth.

Each shirt sells for \\$12 and costs \\$6 to make. Each pair of shorts sells for \\$8 and costs \\$4 to make, while each pair of pants sells for \\$15 and costs \\$8 to make.

Determine which item(s) should be manufactured, and how many of each.

In [2]:
%%gams
set item /shirt, shorts, pants/;
set input /   labor    "hours available per week"
              cloth    "sq yards available per week"/;
set revcost / price    "selling price in dollars"
              var-cost "variable cost in dollars"/;

parameter rentmach(item) "cost of rental in dollars/week"
          / shirt 200, shorts 150, pants 100/;

* Table 2 from Winston
table requirements(item,input)
      labor     cloth
shirt   3         4
shorts  2         3
pants   6         4;

* Table 3 from Winston
table revenue(item,revcost)
        price   var-cost
shirt    12        6
shorts   8         4
pants    15        8;

parameter resources (input) / labor 150, cloth 160/;
scalar bigM /1000/;

positive variable amount(item);
binary variable useMachine(item);
variable profit;

equations EQresources(input), EQrental(item), objective;

EQresources(input)..
        sum(item,requirements(item,input)*amount(item)) =l= resources(input);
    
EQrental(item)..
        amount(item) =l= useMachine(item)*bigM;
    
objective..
        profit =e=   sum(item, revenue(item,'price')*amount(item))
                   - sum(item, revenue(item,'var-cost')*amount(item)) 
                   - sum(item, rentmach(item) * useMachine(item));

model fixedCostExample/all/;

Set the relative optimality tolerance (gap) to 0, then solve bigM problem:

In [3]:
gams.gams('fixedCostExample.optcr=0; solve fixedCostExample using mip maximizing profit;')
print(f"amount =\n {m['amount'].records[['item','level']]}\nuse machines =\n {m['useMachine'].records[['item','level']]}")

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),75.0,6,7,MIP,CPLEX,0.004


amount =
      item  level
0   shirt    0.0
1  shorts    0.0
2   pants   25.0
use machines =
      item  level
0   shirt    0.0
1  shorts    0.0
2   pants    1.0


Use cplex with indicator constraints (turn on EQrental2 if useMachine = 0)

In [4]:
%%gams
$onecho > cplex.opt
indic EQrental2(item)$useMachine(item) 0
$offecho

equations EQrental2(item);

EQrental2(item)..
        amount(item) =e= 0;

model fixedCostExample2/EQresources, EQrental2, objective/;
fixedCostExample2.optfile=1;
fixedCostExample2.optcr=0; 

solve fixedCostExample2 using mip maximizing profit;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),75.0,6,7,MIP,CPLEX,0.003


In [5]:
print(f"amount =\n {m['amount'].records[['item','level']]}\nuse machines =\n {m['useMachine'].records[['item','level']]}")

amount =
      item  level
0   shirt    0.0
1  shorts    0.0
2   pants   25.0
use machines =
      item  level
0   shirt    0.0
1  shorts    0.0
2   pants    1.0


Modify problem to have different costs, capacities and multiple machines

In [6]:
%%gams
parameter rentmach(item) "cost of rental in dollars/week"
          / shirt 30, shorts 50, pants 20/;

parameter capacityMachine(item) "capacity of each machine"
          / shirt 10, shorts 10, pants 8/;
    
integer  variable numberMachines(item);

* No change to form of equations, just data
EQrental(item)..
        amount(item) =l= numberMachines(item)*capacityMachine(item);

objective..
        profit =e=   sum(item, revenue(item,'price')*amount(item))
                   - sum(item, revenue(item,'var-cost')*amount(item)) 
                   - sum(item, rentmach(item) * numberMachines(item));
            
model fixedCostExample /EQresources, EQrental, objective/;

solve fixedCostExample using mip maximizing profit;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),126.0,6,7,MIP,CPLEX,0.003


In [7]:
print(f"amount =\n {m['amount'].records[['item','level']]}\nnumber machines =\n {m['numberMachines'].records[['item','level']]}")

amount =
      item  level
0   shirt   30.0
1  shorts    0.0
2   pants    8.0
number machines =
      item  level
0   shirt    3.0
1  shorts    0.0
2   pants    1.0


In [8]:
%gams_cleanup --closedown